#### Create SP Table (Excel to Excel)
1. Put data table under the same position as the ipynb file
2. Change its name to "Score.xlsx"
3. Run this program
4. New file named "SP.xlsx" will be created as the SP table of the data table
* SP table will be created with sum and percentage

In [84]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, colors, Alignment, Border, Side, PatternFill

#read data
score = pd.read_excel('./Score.xlsx',header=0,index_col=0)
data_row = score.shape[0]
data_col = score.shape[1]
ExcelType = score.copy(deep = True)
for i in range(0,data_row):
    for j in range(0,data_col):
        ExcelType.iloc[i,j]=0
ExcelP = ExcelType.copy(deep = True)


#sum&sort
score['Row_sum'] = score.sum(axis=1)
score.sort_values('Row_sum',ascending = False,inplace = True)
score.loc['Col_sum'] = score.sum()
score.sort_values('Col_sum',axis = 1,ascending = False,inplace = True)
rn = score.pop('Row_sum')
score.insert(score.shape[1],'Row_sum',rn)
#copy for type
ExcelType = score.copy(deep = True)
for i in range(0,data_row):
    for j in range(0,data_col):
        ExcelType.iloc[i,j]=0
ExcelP = ExcelType.copy(deep = True)
#percentage
score.loc['Col_percentage'] = score.apply(lambda x:('%.2f%%')%((x['Col_sum']/data_row)*100),axis=0)
score.at['Col_percentage','Row_sum'] = ""
def rp(x):
    if (x.name == 'Col_percentage'):
        return ('%.2f%%')%((score.at['Col_sum','Row_sum']/(data_col*data_row))*100)
    elif(x. name == 'Col_sum'):
        return ""
    else:
        #return x['Row_sum']/data_col
        return ('%.2f%%')%((x['Row_sum']/data_col)*100)   
score['Row_percentage'] = score.apply(lambda x:rp(x),axis=1)


#write to SP.xlsx
writer = pd.ExcelWriter('sp.xlsx',engine = 'openpyxl')
score.to_excel(writer,sheet_name="SP")
worksheet = writer.sheets['SP']


#create border type
#color
s_blue = '0000FF'
p_red = 'FF0000'
sp_purple = '8A2BE2'
#only s type
bordertype1 = Border(right=Side(border_style='thick', color=s_blue))
bordertype2 = Border(right=Side(border_style='thick', color=s_blue),
                    bottom=Side(border_style='thick', color=s_blue))
bordertype3 = Border(bottom=Side(border_style='thick', color=s_blue))
#only p type
bordertype4 = Border(right=Side(border_style='thick', color=p_red))
bordertype5 = Border(right=Side(border_style='thick', color=p_red),
                    bottom=Side(border_style='thick', color=p_red))
bordertype6 = Border(bottom=Side(border_style='thick', color=p_red))
#s1+p
bordertype7 = Border(right=Side(border_style='thick', color=sp_purple))
bordertype8 = Border(right=Side(border_style='thick', color=sp_purple),
                    bottom=Side(border_style='thick', color=p_red))
bordertype9 = Border(right=Side(border_style='thick', color=s_blue),
                    bottom=Side(border_style='thick', color=p_red))
#s2+p
bordertype10 = Border(right=Side(border_style='thick', color=sp_purple),
                    bottom=Side(border_style='thick', color=s_blue))
bordertype11 = Border(right=Side(border_style='thick', color=sp_purple),
                    bottom=Side(border_style='thick', color=sp_purple))
bordertype12 = Border(right=Side(border_style='thick', color=s_blue),
                    bottom=Side(border_style='thick', color=sp_purple))
#s3+p
bordertype13 = Border(right=Side(border_style='thick', color=p_red),
                    bottom=Side(border_style='thick', color=s_blue))
bordertype14 = Border(right=Side(border_style='thick', color=p_red),
                    bottom=Side(border_style='thick', color=sp_purple))
bordertype15 = Border(bottom=Side(border_style='thick', color=sp_purple))
#merge s&p
def merge_type(s,p):
    if s == 1:
        if p == 4:
            return 7
        elif p == 5:
            return 8
        elif p == 6:
            return 9
    elif s == 2:
        if p == 4:
            return 10
        elif p == 5:
            return 11
        elif p == 6:
            return 12
    elif s == 3:
        if p == 4:
            return 13
        elif p == 5:
            return 14
        elif p == 6:
            return 15

        
#data_row = Col_sum data_col = Row_sum        
#decide s
for r in range(0,data_row):
    rs = ExcelType.iloc[r,data_col]
    if (rs != 0):
        ExcelType.iloc[r,rs-1] = 1
    if (ExcelType.iloc[r,data_col] > ExcelType.iloc[r+1,data_col]):
        down = ExcelType.iloc[r+1,data_col]
        top = ExcelType.iloc[r,data_col]
        for l in range(down,top):
            if (ExcelType.iloc[r,l] == 1):
                ExcelType.iloc[r,l] = 2
            else:
                ExcelType.iloc[r,l] = 3
    elif(ExcelType.iloc[r,data_col] < ExcelType.iloc[r+1,data_col]):
        down = 0
        top = ExcelType.iloc[r,data_col]
        for l in range(down,top):
            if (ExcelType.iloc[r,l] == 1):
                ExcelType.iloc[r,l] = 2
            else:
                ExcelType.iloc[r,l] = 3
#decide p
for l in range(0,data_col):
    cs = score.iloc[data_row,l]
    if (cs !=0):
        ExcelP.iloc[cs-1,l]=6
    if (ExcelP.iloc[data_row,l] > ExcelP.iloc[data_row,l+1]):
        down = ExcelP.iloc[data_row,l+1]
        top = ExcelP.iloc[data_row,l]
        for r in range(down,top):
            if (ExcelP.iloc[r,l] == 6):
                ExcelP.iloc[r,l] = 5
            else:
                ExcelP.iloc[r,l] = 4
    elif (ExcelP.iloc[data_row,l] < ExcelP.iloc[data_row,l+1]):
        down = 0
        top = ExcelP.iloc[data_row,l]
        for r in range(down,top):
            if (ExcelP.iloc[r,l] == 6):
                ExcelP.iloc[r,l] = 5
            else:
                ExcelP.iloc[r,l] = 4
#merage type
for i in range(0,data_row):
    for j in range(0,data_col):
        if (ExcelType.iloc[i,j] != 0 and ExcelP.iloc[i,j]!= 0):
            ExcelType.iloc[i,j]=merge_type(ExcelType.iloc[i,j],ExcelP.iloc[i,j])
        elif (ExcelType.iloc[i,j] == 0 and ExcelP.iloc[i,j]!= 0):
            ExcelType.iloc[i,j]=ExcelP.iloc[i,j]


#load border type onto SP table
def loadtype(et):
    if (et == 1):
        return bordertype1
    elif (et == 2):
        return bordertype2
    elif (et == 3):
        return bordertype3
    elif (et == 4):
        return bordertype4
    elif (et == 5):
        return bordertype5
    elif (et == 6):
        return bordertype6
    elif (et == 7):
        return bordertype7
    elif (et == 8):
        return bordertype8
    elif (et == 9):
        return bordertype9
    elif (et == 10):
        return bordertype10
    elif (et == 11):
        return bordertype11
    elif (et == 12):
        return bordertype12
    elif (et == 13):
        return bordertype13
    elif (et == 14):
        return bordertype14
    elif (et == 15):
        return bordertype15
    else:
        return
for i in range(1,data_row+1):
    for j in range(1,data_col+1):
        worksheet.cell(row=i+1,column=j+1).border = loadtype(ExcelType.iloc[i-1,j-1])

#set head & index style
s_fill = PatternFill(fill_type = 'solid', fgColor = "F0F8FF")
p_fill = PatternFill(fill_type = 'solid', fgColor = "FFE4E1")
c_fill = PatternFill(fill_type = 'solid', fgColor = "E6E6FA")
#load style
cn = 2
for i in range (1,data_row+1):
    worksheet.cell(row = i+1,column = 1).fill = s_fill
for j in range (1,data_col+1):
    worksheet.cell(row = 1,column = j+1).fill = p_fill
for i in range (data_row+1,data_row+1+cn):
    worksheet.cell(row = i+1,column = 1).fill = c_fill
for j in range (data_col+1,data_col+1+cn):
    worksheet.cell(row = 1,column = j+1).fill = c_fill

#output    
writer.save()
print(score)

                    P9      P3      P4      P7      P1      P6      P8  \
S11                  1       1       1       1       1       1       1   
S8                   1       1       1       0       1       1       1   
S4                   1       1       0       1       1       1       1   
S2                   1       0       1       1       1       0       0   
S5                   1       1       1       1       0       0       0   
S6                   1       0       1       1       0       1       1   
S10                  1       0       1       1       0       1       1   
S1                   1       1       1       0       1       0       0   
S9                   1       1       0       1       1       0       0   
S3                   1       1       0       0       0       0       0   
S7                   1       0       0       0       0       0       0   
S12                  0       0       0       0       0       0       0   
Col_sum             11       7       7